In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import scipy.interpolate
from astropy.io import fits
from matplotlib.colors import SymLogNorm, Normalize
from PIL import Image

In [2]:
original = fits.open('../../2019/12/bgmodel.fits')[0].data
original.shape

(24, 24)

In [3]:
plt.figure(figsize=(10,10))
plt.imshow(original, cmap='Greys_r')
plt.colorbar()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 1D interpolation

In [81]:
def interpolate(y, newsize):
    x = np.linspace(0, 1, len(y))
    newx = np.linspace(0, 1, newsize)
    return scipy.interpolate.interp1d(x, y, 'cubic')(newx)

In [82]:
def myinterpolate(y, newsize):
    # https://en.wikiversity.org/wiki/Cubic_Spline_Interpolation
    # For inversion: https://en.wikipedia.org/wiki/Tridiagonal_matrix
    size = len(y)
    result = np.zeros(newsize)
    
    # x = [0:n:1]
    newx = np.linspace(0, len(y)-1, newsize)
    
    # h[i] = x[i] - x[i-1], but since pixels are equally spaced, all h_i are 1, so we skip them
    # u[i] = h[i] / (h[i] + h[i+1]), which is always 1/2
    # l[i] = 1 - u[i], which, again, is always 1/2
    u = 0.5
    l = 0.5
       
    d = np.zeros(size)
    # By definition of natural cubic spline, second derivative is 0
    d[0] = d[-1] = 0.
    # X is equally spaced with width 1, so x[i] - x[i-1] 
    d[1:-1] = 6 * ((y[2:] - y[1:-1]) - (y[1:-1] - y[:-2])) / 2
    
    # From https://en.wikiversity.org/wiki/Cubic_Spline_Interpolation
    # We now use a matrix A, where the diagonal is 2, the lower diagonal is u[n], and the upper diagonal l[n]
    # The rest is 0
    # Therefore, we can just apply the Tridiagonal matrix algorithm to solve the system
    # A x M = d
    # We do not need the full matrix. As a matter of fact, we do not even need arrays, as all $\lambda_i$ and $\mu_i$ have the same
    # values except if i = 0
    # @see https://en.wikipedia.org/wiki/Tridiagonal_matrix_algorithm
    
    # Forward sweep
    M = np.zeros(size)
    B = np.full(size, 2.)
    for i in range(1, size):
        a = u if i < size - 1 else 0
        c = l if i > 1 else 0
        W = a / B[i-1]
        B[i] = B[i] - W * c
        d[i] = d[i] - W * d[i-1]

    # Back substitution
    M[-1] = d[-1] / B[-1]
    for i in range(size - 2, -1, -1):
        c = l if i > 1 else 0
        M[i] = (d[i] - c * M[i + 1]) / B[i]
    
    # M is always divided by 6 on formula 1 since h[i] is 1
    M /= 6
    
    for ni, nx in enumerate(newx):
        # i is a proxy for x[i]
        i = int(nx) + 1
        if i > size-1:
            i = size-1
        result[ni] =   M[i-1] * (i - nx)**3 \
                    + M[i]  * (nx - (i-1))**3 \
                    + (y[i-1] - M[i-1]) * (i - nx) \
                    + (y[i] - M[i]) * (nx - (i-1))
        
    return result

In [83]:
data_y = original[13,:]
data_x = np.arange(len(data_y))

In [84]:
scinew = interpolate(data_y, 100)
mynew = myinterpolate(data_y, 100)

new_x = np.linspace(0, len(data_y)-1, 100)

In [85]:
plt.close()

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(20,15), sharex=True)
plt.subplots_adjust(hspace=0)
axes.plot(data_x, data_y, marker='o', label='Downsampled', c='orange')
axes.plot(new_x, scinew, marker='x', label='SciPy', c='r')
axes.plot(new_x, mynew, marker='.', label='My', c='blue', linestyle=':')

axes.legend()

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 2D interpolation

In [86]:
cell_size = 64
newsize = original.shape[0]*cell_size, original.shape[1]*cell_size

In [87]:
ox = np.arange(original.shape[0])
oy = np.arange(original.shape[1])
nx = np.arange(newsize[0])/(newsize[0]-1) * (original.shape[0]-1)
ny = np.arange(newsize[1])/(newsize[1]-1) * (original.shape[1]-1)
sample = scipy.interpolate.interp2d(ox, oy, original, kind='cubic')(nx, ny)

In [88]:
img = np.zeros(newsize)
for i in range(original.shape[1]):
    img[:, i*cell_size] = myinterpolate(original[:, i], newsize[0])
for i in range(img.shape[0]):
    img[i, :] = myinterpolate(img[i, ::cell_size], newsize[1])

In [89]:
fig, axes = plt.subplots(ncols=3, nrows=1, figsize=(22, 10))
axes[0].imshow(img, cmap='Greys_r')
axes[1].imshow(sample, cmap='Greys_r')
d = axes[2].imshow(img-sample)
plt.colorbar(d)
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [90]:
from astropy.io import fits

In [91]:
fits.PrimaryHDU(img).writeto('bg_scaled.fits', overwrite=True)